In [5]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import itertools

# Tasks 1-4


# Task 1: Permutations and Combinations

In the Lady Tasting Tea experiment, we have twelve cups of tea: six with milk poured first and six with tea poured first. A person claims they can distinguish the order by taste. To test this, we ask them to identify which six cups had milk poured first.<br>
1. Calculate the probability that they correctly identify all six cups by random guessing, assuming they have no special ability.

2. One error. Calculate the probability that they identify at least five of the six correct cups by random guessing.
3. Are two errors acceptable?

In [6]:
# Initial conditions of the experiment:

total_cups = 12
with_milk_first = 6
with_tea_first = 6


In [7]:
# Calculation the number of ways to choose six cups with milk first out of twelve cups:

# https://www.w3schools.com/python/ref_math_comb.asp

# The function calculates all the possible combinations. 
# The order does not matter and the elements can't be selected twice.

total_combinations = math.comb(total_cups,with_milk_first)

total_combinations

924